## Overview 
The dataset we chose contains car evaluation data derived from a hierarchical decision model developed initially for a demonstration of a decision making model and can be found at https://archive.ics.uci.edu/ml/datasets/Car+Evaluation. The authors The dataset contains 6 attributes related to either price or technical characteristics. The 7th attribute represents the estimated class of the car and is based on all other attributes. The dataset consists of 1728 entries and is stripped of structural attributes, which means all attributes are directly related to the estimated car class attribute. There are also three intermediate attributes – PRICE, TECH and COMFORT – which are related to the 6 main attributes.  
## Use case
Choosing a vehicle to purchase can be a tedious process that involves hours of research, with studies showing American drivers spend an average of around 15 hours between realizing the need for a new car and making the purchase [1]. 60% of this time is usually spent in online research of specifications and availability. Generally, although the most important attribute of a car is its ability to transport, the final decision is very often based on an amalgam of its price, safety and capacity. The main purpose for the collection of the dataset we chose was to “actively support the decision maker in the knowledge acquisition and evaluation stages of the decision making process” [2]. 
## Prediction task
The dataset uses a simple hierarchical model to classify cars in one of 4 categories: Unacceptable (unacc), Acceptable (acc), Good (good), Very Good (vgood). The criteria tree is displayed below. The goal of our prediction task is to correctly identify the class associated with the car based on the 6 attributes that are used in the evaluation model, without specifying the model structure itself. 
This could be useful in many different scenarios, such as online-based automotive research and shopping websites. Those often imply mathematical algorithms to present users with a nominal rating ranging 

 

<img src='tree.png' label="Criteria tree"/ height=500 width=500>
https://www.elephant.com/blog/car-insurance/new-study-details-how-long-it-takes-before-car-shoppers-buy 
http://kt.ijs.si/MarkoBohanec/pub/Avignon88.pdf



In [3]:
#Imports
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from scipy.special import expit


df = pd.read_csv('car.data')
buying_maint_map = {'vhigh':3,'high':2,'med':1,'low':0}
df['buy_price'] = df['buy_price'].map(buying_maint_map).astype(np.int)
df['maint_price'] = df['maint_price'].map(buying_maint_map).astype(np.int)
doors_map = {'2':0,'3':1,'4':2,'5more':3}
df['doors'] = df['doors'].map(doors_map).astype(np.int)
persons_map = {'2':0,'3':1,'4':2,'more':3}
df['persons'] = df['persons'].map(persons_map).astype(np.int)
trunk_map = {'small':0,'med':1,'big':2}
df['trunk_size'] = df['trunk_size'].map(trunk_map).astype(np.int)
safety_map = {'low':0,'med':1,'high':2}
df['safety'] = df['safety'].map(safety_map).astype(np.int)
class_map = {'unacc':0,'acc':1,'good':2,'vgood':3}
df['class'] = df['class'].map(class_map).astype(np.int)


feature_cols = ['buy_price','maint_price','doors','persons','trunk_size','safety']
class_cols = ['class']

#Make X a 2D numpy array
X = df[feature_cols].as_matrix()
#Make y a 1D numpy array
y = (df[class_cols]==0).astype(np.int).values.ravel()
y_not_binary = (df[class_cols]).astype(np.int).values.ravel()


In [241]:
#Taken from notebook 6
class BinaryLogisticRegression:
    def __init__(self, eta, iterations=20, C=0.001,l1_norm=0.5):
        self.eta = eta
        self.iters = iterations
        self.C = C
        if ((l1_norm > 1 or l1_norm < 0) and l1_norm != -1):
            raise ValueError("L1 Norm must be between 0 and 1 or -1")
        self.l1_norm = l1_norm
        # internally we will store the weights as self.w_ to keep with sklearn conventions
        
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'Binary Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained Binary Logistic Regression Object'
        
    # convenience, private:
    @staticmethod
    def _add_bias(X):
        return np.hstack((np.ones((X.shape[0],1)),X)) # add bias term
    
    @staticmethod
    def _sigmoid(theta):
        # increase stability, redefine sigmoid operation
        return expit(theta) #1/(1+np.exp(-theta))
    
    # vectorized gradient calculation with regularization using L2 Norm
    def _get_gradient(self,X,y):
        ydiff = y-self.predict_proba(X,add_bias=False).ravel() # get y difference
        gradient = np.mean(X * ydiff[:,np.newaxis], axis=0) # make ydiff a column vector and multiply through
        gradient = gradient.reshape(self.w_.shape)
        if (self.l1_norm == -1):
            return gradient
        return self._regularize(gradient)
    
    def _regularize(self,gradient):
        #Implementation for Elastic Net regularization 
        sub_1 = np.copy(gradient[1:])
        sub_2 = np.copy(gradient[1:])
        
        #Calculate L1 Norm 
        mask = np.logical_and(sub_1 >= (-self.C/2),sub_1 <= (self.C/2))
        sub_1[mask] = 0
        sub_1[sub_1 < (-self.C/2)] += (self.C / 2)
        sub_1[sub_1 > (self.C/2)] -= (self.C / 2)
        
        
        #Calculate L2 Norm
        sub_2 += -2 * self.w_[1:] * self.C
        
        #Combine the regularizations to make an elastic net.
        gradient[1:] = self.l1_norm * sub_1 + (1-self.l1_norm) * sub_2
        return gradient
        
    
    # public:
    def predict_proba(self,X,add_bias=True):
        # add bias term if requested
        Xb = self._add_bias(X) if add_bias else X
        return self._sigmoid(Xb @ self.w_) # return the probability y=1
    
    def predict(self,X):
        return (self.predict_proba(X)>0.5) #return the actual prediction
    
    
    def fit(self, X, y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        
        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            self.w_ += gradient*self.eta # multiply by learning rate 

https://www.coursera.org/learn/ml-regression/lecture/SeZsT/coordinate-descent-for-lasso-normalized-features
Coursera link used to help derive the formula for L1 regularization.

In [300]:
%%time
class StochasticLogisticRegression(BinaryLogisticRegression):
    # stochastic gradient calculation 
    def _get_gradient(self,X,y):
        idx = int(np.random.rand()*len(y)) # grab random instance
        ydiff = y[idx]-self.predict_proba(X[idx],add_bias=False) # get y difference (now scalar)
        gradient = X[idx] * ydiff[:,np.newaxis] # make ydiff a column vector and multiply through
        gradient = gradient.reshape(self.w_.shape)
        if (self.l1_norm == -1):
            return gradient
        return self._regularize(gradient)
    
    
slr = StochasticLogisticRegression(0.1,1000, C=0.001,l1_norm=1) # take a lot more steps!!

slr.fit(X,y)

yhat = slr.predict(X)
print(slr)
print('Accuracy of: ',accuracy_score(y,yhat))

Binary Logistic Regression Object with coefficients:
[[ 3.10526841]
 [ 1.26327965]
 [ 1.14966388]
 [ 0.0206099 ]
 [-1.26879973]
 [-0.39749817]
 [-2.11235392]]
Accuracy of:  0.8651620370370371
Wall time: 50.9 ms


In [303]:
%%time
class NewtonLogisticRegression(BinaryLogisticRegression):
    def _hessian(self,X,y):
        g = self.predict_proba(X,add_bias=False).ravel()
        return X.T @ np.diag(g*(1-g)) @ X - 2 * self.C 
    def fit(self,X,y):
        Xb = self._add_bias(X) # add bias term
        num_samples, num_features = Xb.shape
        self.w_ = np.zeros((num_features,1)) # init weight vector to zeros
        # for as many as the max iterations
        for _ in range(self.iters):
            gradient = self._get_gradient(Xb,y)
            inv_hessian = np.linalg.inv(self._hessian(Xb,y))
            self.w_ +=  inv_hessian@gradient*self.eta # multiply by learning rate 
            
nlr = NewtonLogisticRegression(0.1,10, C=0.001) # take a lot more steps!!
nlr.fit(X,y)

yhat = nlr.predict(X)
print(nlr)
print('Accuracy of: ',accuracy_score(y,yhat))

Binary Logistic Regression Object with coefficients:
[[ 1.38419198e-03]
 [ 1.93458401e-04]
 [ 1.72030526e-04]
 [-5.93905279e-05]
 [-3.67708094e-04]
 [-1.65594500e-04]
 [-6.01517508e-04]]
Accuracy of:  0.8929398148148148
Wall time: 178 ms


In [304]:
class MultiClassLogisticRegression:
    def __init__(self, eta, iterations=20, C=0.0001, optimize_func='steepest',l1_norm=0.5):
        self.eta = eta
        self.iters = iterations
        self.C = C
        self.classifiers_ = []
        self.optimize_func = optimize_func
        self.l1_norm = l1_norm
        # internally we will store the weights as self.w_ to keep with sklearn conventions
    
    def __str__(self):
        if(hasattr(self,'w_')):
            return 'MultiClass Logistic Regression Object with coefficients:\n'+ str(self.w_) # is we have trained the object
        else:
            return 'Untrained MultiClass Logistic Regression Object'
        
    def fit(self,X,y):
        num_samples, num_features = X.shape
        self.unique_ = np.sort(np.unique(y)) # get each unique class value
        num_unique_classes = len(self.unique_)
        self.classifiers_ = []
        for i,yval in enumerate(self.unique_): # for each unique value
            y_binary = (y==yval).astype(np.int) # create a binary problem
            # train the binary classifier for this class
            if self.optimize_func == 'stochastic':
                 hblr = StochasticLogisticRegression(self.eta,self.iters,self.C,l1_norm = self.l1_norm)
            elif self.optimize_func == 'steepest':
                hblr = BinaryLogisticRegression(self.eta,self.iters,self.C,l1_norm = self.l1_norm)
            elif self.optimize_func == 'newton':
                hblr = NewtonLogisticRegression(self.eta,self.iters,self.C,l1_norm = self.l1_norm)
            hblr.fit(X,y_binary)
            # add the trained classifier to the list
            self.classifiers_.append(hblr)
            
        # save all the weights into one matrix, separate column for each class
        self.w_ = np.hstack([x.w_ for x in self.classifiers_]).T
        
    def predict_proba(self,X):
        probs = []
        for hblr in self.classifiers_:
            probs.append(hblr.predict_proba(X).reshape((len(X),1))) # get probability for each classifier
        
        return np.hstack(probs) # make into single matrix
    
    def predict(self,X):
        return np.argmax(self.predict_proba(X),axis=1) # take argmax along row
    



In [394]:
%%time
lr = MultiClassLogisticRegression(0.1,iterations=250,C=0.00001,optimize_func='steepest')
lr.fit(X,y_not_binary)
print(lr)

yhat = lr.predict(X)
print('Accuracy of: ',accuracy_score(y_not_binary,yhat))

MultiClass Logistic Regression Object with coefficients:
[[ 1.07241361  0.86258341  0.80408431  0.14023905 -0.82246482 -0.0731898
  -1.21356912]
 [-1.16057672 -0.36071249 -0.37714207 -0.18977163  0.4727244  -0.15508043
   0.64756361]
 [-0.52221446 -0.99670015 -0.99670015 -0.26945457  0.11810753 -0.27441964
   0.00443686]
 [-0.70688151 -1.02877499 -0.80201932 -0.2993838  -0.02292801 -0.02129836
   0.22804441]]
Accuracy of:  0.7644675925925926
Wall time: 232 ms


In [411]:
%%time
lr_st = MultiClassLogisticRegression(0.1,iterations=1000,C=0.00001,optimize_func='stochastic')
lr_st.fit(X,y_not_binary)
print(lr_st)

yhat_st = lr_st.predict(X)
print('Accuracy of: ',accuracy_score(y_not_binary,yhat_st))

MultiClass Logistic Regression Object with coefficients:
[[ 3.43376064  0.91109925  1.05825449  0.14149201 -1.64407873 -0.27249408
  -1.98629881]
 [-3.13606038 -0.41059259 -0.49080561 -0.01677221  0.75234871  0.15536722
   0.82116911]
 [-1.19891464 -1.38832865 -1.82712731 -0.34378097  0.20434965 -0.66132599
   0.63820601]
 [-1.81059544 -1.821184   -1.34306785 -0.57224608 -0.10109098  0.11113448
   0.63007493]]
Accuracy of:  0.78125
Wall time: 147 ms


In [404]:
%%time
lr = MultiClassLogisticRegression(0.1,iterations=2,C=0.01,optimize_func='newton',l1_norm=0)
lr.fit(X,y_not_binary)
print(lr)

yhat = lr.predict(X)
print('Accuracy of: ',accuracy_score(y_not_binary,yhat))

MultiClass Logistic Regression Object with coefficients:
[[ 2.76030519e-04  3.87913330e-05  3.45047708e-05 -1.17901003e-05
  -7.34861502e-05 -3.29528930e-05 -1.20157500e-04]
 [-3.06442836e-04 -6.21252737e-06 -7.49849601e-06  7.07581525e-06
   5.40827949e-05  1.56727174e-05  8.19803680e-05]
 [-1.92044544e-04 -1.72496597e-05 -1.72496597e-05  9.68229429e-07
   9.64878935e-06  1.20559365e-06  1.20559365e-05]
 [-2.40568032e-04 -1.53205723e-05 -9.74804155e-06  3.75462921e-06
   9.76375201e-06  1.60745820e-05  2.61211957e-05]]
Accuracy of:  0.7864583333333334
Wall time: 132 ms


In [491]:
import copy
c_list = [0.00001,0.0001,0.001,0.01,0.1,1,10,100,1000]
optimization_list = ['newton','stochastic','steepest']
regularization_list = [-1,0,1]
eta = 0.1

iterations = {
    'newton':3,
    'stochastic':1000,
    'steepest':1000
}

tests = dict.fromkeys(regularization_list,dict.fromkeys(optimization_list,dict.fromkeys(c_list,"")))
results = copy.deepcopy(tests)
weights = copy.deepcopy(tests)


for regularization in tests.keys():
    for optimization in tests[regularization].keys():
        for c in tests[regularization][optimization].keys():
            tests[regularization][optimization][c] = MultiClassLogisticRegression(eta,iterations=iterations[optimization],C=int(c),optimize_func=optimization,l1_norm=int(regularization))

In [490]:
%%time
print
for regularization in tests.keys():
    for optimization in tests[regularization].keys():
        for c in tests[regularization][optimization].keys():
            test = tests[regularization][optimization][c]
            test.fit(X,y_not_binary)
            #print(test)
            yhat = test.predict(X)
            score = accuracy_score(y_not_binary,yhat)
            print(regularization,',',optimization,',',c,': Accuracy of: ',score)
            results[regularization][optimization][c] = score
            weights[regularization][optimization][c] = test.w_
        print()
        print()


-1 , newton , 1e-05 : Accuracy of:  0.7841435185185185
-1 , newton , 0.0001 : Accuracy of:  0.7841435185185185
-1 , newton , 0.001 : Accuracy of:  0.7841435185185185
-1 , newton , 0.01 : Accuracy of:  0.7841435185185185
-1 , newton , 0.1 : Accuracy of:  0.7841435185185185
-1 , newton , 1 : Accuracy of:  0.7002314814814815
-1 , newton , 10 : Accuracy of:  0.7002314814814815
-1 , newton , 100 : Accuracy of:  0.7002314814814815
-1 , newton , 1000 : Accuracy of:  0.7002314814814815


-1 , stochastic , 1e-05 : Accuracy of:  0.7841435185185185
-1 , stochastic , 0.0001 : Accuracy of:  0.7841435185185185
-1 , stochastic , 0.001 : Accuracy of:  0.7841435185185185
-1 , stochastic , 0.01 : Accuracy of:  0.7841435185185185
-1 , stochastic , 0.1 : Accuracy of:  0.7841435185185185
-1 , stochastic , 1 : Accuracy of:  0.7002314814814815
-1 , stochastic , 10 : Accuracy of:  0.7002314814814815
-1 , stochastic , 100 : Accuracy of:  0.7002314814814815
-1 , stochastic , 1000 : Accuracy of:  0.7002314814814